# Model 4 (Lample 2016): BiLSTM-CRF with Char-BiLSTM

**This notebook EXACTLY replicates Lample et al. (2016) hyperparameters**

**Paper**: "Neural Architectures for Named Entity Recognition" (NAACL 2016)

**Original Results on CoNLL-2003**: 90.94% F1

## Key Difference from Ma & Hovy (2016):

**Character Encoding:**
- **Lample (2016)**: Character-level **BiLSTM** (forward + backward LSTM over characters)
- **Ma & Hovy (2016)**: Character-level **CNN** (convolution + max pooling)

Both papers use BiLSTM + CRF at the word level, but differ in how they encode characters.

## Hyperparameters from Paper:

| Component | Lample et al. (2016) |
|-----------|---------------------|
| Char encoding | **BiLSTM** (25d hidden per direction) |
| Word LSTM hidden | **100** per direction |
| Optimizer | **SGD + momentum** |
| Learning rate | **0.01** |
| Dropout | **0.5** |
| Gradient clipping | **5.0** |

**Reference**: Lample, G., Ballesteros, M., Subramanian, S., Kawakami, K., & Dyer, C. (2016). Neural architectures for named entity recognition. arXiv preprint arXiv:1603.01360.

## 1. Setup and Imports

In [16]:
# Install required packages
import sys
print(f"Python: {sys.executable}")

!{sys.executable} -m pip install torch pytorch-crf gensim tqdm

print("\n✅ Packages installed!")

Python: /usr/local/bin/python3

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

✅ Packages installed!


In [17]:
import json
import numpy as np
import pickle
import time
from collections import Counter
import os

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# CRF
from torchcrf import CRF

# Embeddings
import gensim.downloader as api

# Progress bar
from tqdm import tqdm

# Our evaluation utilities
from utils import print_evaluation_report, evaluate_entity_spans

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

print("Imports successful!")

Using device: cpu
Imports successful!


## 2. Load Data

In [18]:
def load_jsonl(file_path):
    """Load JSONL file"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

# Load data
train_data = load_jsonl('train_split.jsonl')
val_data = load_jsonl('val_split.jsonl')

print(f"Training samples: {len(train_data):,}")
print(f"Validation samples: {len(val_data):,}")

# Extract tokens and tags
train_tokens = [sample['tokens'] for sample in train_data]
train_tags = [sample['ner_tags'] for sample in train_data]

val_tokens = [sample['tokens'] for sample in val_data]
val_tags = [sample['ner_tags'] for sample in val_data]

Training samples: 90,320
Validation samples: 10,036


## 3. Build Vocabularies

**Lample (2016) settings:**
- Word frequency threshold: Not specified (we'll use 5 like Ma & Hovy)
- Character embedding: 25d

In [19]:
# Build word vocabulary
word_counts = Counter()
for tokens in train_tokens:
    word_counts.update(tokens)

MIN_WORD_FREQ = 5
word2idx = {'<PAD>': 0, '<UNK>': 1}
for word, count in word_counts.items():
    if count >= MIN_WORD_FREQ:
        word2idx[word] = len(word2idx)

idx2word = {idx: word for word, idx in word2idx.items()}
vocab_size = len(word2idx)

print(f"Word vocabulary size: {vocab_size:,} (min freq = {MIN_WORD_FREQ})")

Word vocabulary size: 15,092 (min freq = 5)


In [20]:
# Build character vocabulary
char2idx = {'<PAD>': 0, '<UNK>': 1}

chars = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 .,!?:;\'"()-[]{}@#$%^&*+=/<>\\|`~_'
for char in chars:
    if char not in char2idx:
        char2idx[char] = len(char2idx)

idx2char = {idx: char for char, idx in char2idx.items()}
char_vocab_size = len(char2idx)

print(f"Character vocabulary size: {char_vocab_size}")

Character vocabulary size: 97


In [21]:
# Build tag vocabulary
tag2idx = {}
for tags in train_tags:
    for tag in tags:
        if tag not in tag2idx:
            tag2idx[tag] = len(tag2idx)

idx2tag = {idx: tag for tag, idx in tag2idx.items()}
num_tags = len(tag2idx)

print(f"Number of NER tags: {num_tags}")
print(f"Tags: {list(tag2idx.keys())}")

Number of NER tags: 15
Tags: ['O', 'B-ORG', 'I-ORG', 'B-Facility', 'I-Facility', 'B-OtherPER', 'I-OtherPER', 'B-Politician', 'I-Politician', 'B-HumanSettlement', 'I-HumanSettlement', 'B-Artist', 'I-Artist', 'B-PublicCorp', 'I-PublicCorp']


## 4. Load Pre-trained Word Embeddings (GloVe)

**Lample (2016) uses GloVe 100d**

In [22]:
print("Downloading GloVe embeddings...")
glove_model = api.load('glove-wiki-gigaword-100')

EMBEDDING_DIM = 100
print(f"GloVe embeddings loaded! Dimension: {EMBEDDING_DIM}")

GloVe embeddings loaded! Dimension: 100


In [23]:
# Create embedding matrix
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

found = 0
for word, idx in word2idx.items():
    if word in ['<PAD>', '<UNK>']:
        continue
    
    try:
        embedding_matrix[idx] = glove_model[word.lower()]
        found += 1
    except KeyError:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(EMBEDDING_DIM,))

# Special tokens
embedding_matrix[word2idx['<PAD>']] = np.zeros(EMBEDDING_DIM)
embedding_matrix[word2idx['<UNK>']] = np.random.normal(scale=0.6, size=(EMBEDDING_DIM,))

print(f"Words found in GloVe: {found:,} / {vocab_size:,} ({found/vocab_size*100:.1f}%)")

Words found in GloVe: 14,897 / 15,092 (98.7%)


## 5. Dataset Class

In [24]:
class NERDataset(Dataset):
    def __init__(self, tokens_list, tags_list, word2idx, char2idx, tag2idx):
        self.tokens_list = tokens_list
        self.tags_list = tags_list
        self.word2idx = word2idx
        self.char2idx = char2idx
        self.tag2idx = tag2idx
    
    def __len__(self):
        return len(self.tokens_list)
    
    def __getitem__(self, idx):
        tokens = self.tokens_list[idx]
        tags = self.tags_list[idx]
        
        # Words
        word_ids = [self.word2idx.get(token, self.word2idx['<UNK>']) for token in tokens]
        
        # Characters (variable length - will be padded in collate_fn)
        char_ids = []
        for token in tokens:
            chars = [self.char2idx.get(c, self.char2idx['<UNK>']) for c in token]
            char_ids.append(chars)
        
        # Tags
        tag_ids = [self.tag2idx[tag] for tag in tags]
        
        return {
            'word_ids': word_ids,
            'char_ids': char_ids,  # List of lists (variable length)
            'tag_ids': tag_ids,
            'length': len(tokens)
        }

train_dataset = NERDataset(train_tokens, train_tags, word2idx, char2idx, tag2idx)
val_dataset = NERDataset(val_tokens, val_tags, word2idx, char2idx, tag2idx)

print(f"Train dataset: {len(train_dataset)} samples")
print(f"Val dataset: {len(val_dataset)} samples")

Train dataset: 90320 samples
Val dataset: 10036 samples


In [25]:
# Collate function for character BiLSTM - filters empty sequences
def collate_fn(batch):
    """Custom collate for variable-length characters - filters empty sequences"""
    # Sort by length
    batch = sorted(batch, key=lambda x: x['length'], reverse=True)
    
    # Filter out empty sequences BEFORE processing
    batch = [item for item in batch if item['length'] > 0]
    
    # Handle completely empty batch
    if len(batch) == 0:
        return {
            'word_ids': torch.zeros((0, 0), dtype=torch.long),
            'char_ids': torch.zeros((0, 0), dtype=torch.long),
            'char_lens': [],
            'tag_ids': torch.zeros((0, 0), dtype=torch.long),
            'lengths': [],
            'mask': torch.zeros((0, 0), dtype=torch.bool)
        }
    
    word_ids = [torch.LongTensor(item['word_ids']) for item in batch]
    tag_ids = [torch.LongTensor(item['tag_ids']) for item in batch]
    lengths = [item['length'] for item in batch]
    
    # Pad word and tag sequences
    word_ids_padded = pad_sequence(word_ids, batch_first=True, padding_value=word2idx['<PAD>'])
    tag_ids_padded = pad_sequence(tag_ids, batch_first=True, padding_value=tag2idx['O'])
    
    # For characters: need to pad each word's character sequence
    # char_ids is list of lists of lists: batch -> words -> chars
    char_ids_all = []  # Will store all character sequences flattened
    char_lens_all = []  # Length of each character sequence
    
    for item in batch:
        for word_chars in item['char_ids']:
            char_ids_all.append(torch.LongTensor(word_chars))
            char_lens_all.append(len(word_chars))
    
    # Pad character sequences
    char_ids_padded = pad_sequence(char_ids_all, batch_first=True, padding_value=char2idx['<PAD>'])
    
    # Create mask
    max_len = word_ids_padded.size(1)
    batch_size = len(batch)
    mask = torch.zeros((batch_size, max_len), dtype=torch.bool)
    for i, length in enumerate(lengths):
        mask[i, :length] = True
    
    return {
        'word_ids': word_ids_padded,
        'char_ids': char_ids_padded,
        'char_lens': char_lens_all,
        'tag_ids': tag_ids_padded,
        'lengths': lengths,
        'mask': mask
    }

# Create DataLoaders
BATCH_SIZE = 10  # Following paper settings

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn
)

print(f"Train batches: {len(train_loader)} (batch size = {BATCH_SIZE})")
print(f"Val batches: {len(val_loader)}")

Train batches: 9032 (batch size = 10)
Val batches: 1004


## 6. Model Architecture

**Key difference: Character-level BiLSTM (not CNN)**

```
Characters → Char Embedding (25d) → Char BiLSTM (25 hidden × 2) → 50d
Words → Word Embedding (100d GloVe)
    ↓
Concat (150d) → Word BiLSTM (100 hidden × 2) → 200d
    ↓
Dropout (0.5) → Linear → CRF
```

In [26]:
class CharBiLSTM(nn.Module):
    """Character-level BiLSTM (Lample et al. 2016)"""
    def __init__(self, char_vocab_size, char_emb_dim, char_hidden_dim):
        super().__init__()
        self.char_embedding = nn.Embedding(char_vocab_size, char_emb_dim, padding_idx=0)
        
        # BiLSTM over characters
        self.char_lstm = nn.LSTM(
            input_size=char_emb_dim,
            hidden_size=char_hidden_dim,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )
        
    def forward(self, char_ids, char_lens):
        """
        Args:
            char_ids: (total_words, max_char_len) - all words in batch flattened
            char_lens: list of character lengths for each word
        Returns:
            char_features: (total_words, char_hidden_dim * 2)
        """
        # Embed characters
        char_embeds = self.char_embedding(char_ids)  # (total_words, max_char_len, char_emb_dim)
        
        # Pack padded sequence for efficiency
        char_lens_tensor = torch.LongTensor(char_lens)
        
        # Clamp lengths to at least 1 to avoid pack_padded_sequence errors
        char_lens_clamped = torch.clamp(char_lens_tensor, min=1)
        
        packed = pack_padded_sequence(
            char_embeds, char_lens_clamped.cpu(), 
            batch_first=True, enforce_sorted=False
        )
        
        # BiLSTM
        packed_output, (hidden, _) = self.char_lstm(packed)
        
        # Take final hidden states from both directions
        # hidden: (2, total_words, char_hidden_dim) -> (total_words, char_hidden_dim * 2)
        char_features = torch.cat([hidden[0], hidden[1]], dim=-1)
        
        return char_features


class BiLSTM_CRF_Lample2016(nn.Module):
    """BiLSTM-CRF with Char-BiLSTM (Lample et al. 2016)"""
    def __init__(self, vocab_size, char_vocab_size, embedding_dim, char_emb_dim,
                 char_hidden_dim, lstm_hidden_dim, num_tags, embedding_matrix=None):
        super().__init__()
        
        # Word embeddings
        self.word_embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        if embedding_matrix is not None:
            self.word_embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
            self.word_embedding.weight.requires_grad = True
        
        # Character BiLSTM
        self.char_bilstm = CharBiLSTM(char_vocab_size, char_emb_dim, char_hidden_dim)
        self.char_output_dim = char_hidden_dim * 2
        
        # Word-level BiLSTM
        self.lstm = nn.LSTM(
            input_size=embedding_dim + (char_hidden_dim * 2),  # word + char features
            hidden_size=lstm_hidden_dim,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )
        
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(lstm_hidden_dim * 2, num_tags)
        
        # CRF
        self.crf = CRF(num_tags, batch_first=True)
    
    def forward(self, word_ids, char_ids, char_lens, tags=None, mask=None):
        """
        Args:
            word_ids: (batch, seq_len)
            char_ids: (total_words, max_char_len) - flattened
            char_lens: list of char lengths (length = total_words)
            tags: (batch, seq_len)
            mask: (batch, seq_len)
        """
        batch_size, seq_len = word_ids.size()
        
        # Word embeddings
        word_embeds = self.word_embedding(word_ids)  # (batch, seq_len, word_emb_dim)
        
        # Character features for all words
        char_features = self.char_bilstm(char_ids, char_lens)  # (total_words, char_hidden*2)
        
        # Reshape char features to match word embeddings
        # The char_features are ordered as: word[0,0], word[0,1], ..., word[0,seq_len-1], word[1,0], ...
        # We need to split them back by checking the actual total
        total_words = char_features.size(0)
        expected_words = batch_size * seq_len
        
        # If there are fewer char features than expected (due to variable length sequences),
        # we need to be more careful about how we reshape
        if total_words == expected_words:
            # Simple case: just reshape
            char_features_batched = char_features.view(batch_size, seq_len, self.char_output_dim)
        else:
            # Complex case: we have variable length sequences, need to manually assign
            # This shouldn't happen with our collate_fn that pads, but let's handle it
            char_features_batched = torch.zeros(
                batch_size, seq_len, self.char_output_dim,
                device=char_features.device, dtype=char_features.dtype
            )
            
            char_idx = 0
            for b in range(batch_size):
                for s in range(seq_len):
                    if char_idx < total_words:
                        char_features_batched[b, s] = char_features[char_idx]
                        char_idx += 1
        
        # Concatenate word and character features
        combined = torch.cat([word_embeds, char_features_batched], dim=-1)
        
        # Word-level BiLSTM
        lstm_out, _ = self.lstm(combined)
        lstm_out = self.dropout(lstm_out)
        
        # Emission scores
        emissions = self.fc(lstm_out)
        
        if tags is not None:
            loss = -self.crf(emissions, tags, mask=mask, reduction='mean')
            return loss
        else:
            predictions = self.crf.decode(emissions, mask=mask)
            return predictions

print("Model architecture defined (Lample 2016 - Char BiLSTM)!")

Model architecture defined (Lample 2016 - Char BiLSTM)!


## 7. Initialize Model

**Lample (2016) hyperparameters:**
- Char embedding: **25d**
- Char BiLSTM hidden: **25** per direction (50 total)
- Word BiLSTM hidden: **100** per direction (200 total)
- Optimizer: **SGD + momentum**
- Learning rate: **0.01**

In [27]:
# LAMPLE (2016) HYPERPARAMETERS
CHAR_EMB_DIM = 25       # Paper: 25d
CHAR_HIDDEN_DIM = 25    # Paper: 25 (bidirectional = 50 total)
LSTM_HIDDEN_DIM = 100   # Paper: 100 (bidirectional = 200 total)
LEARNING_RATE = 0.01    # Paper: 0.01
NUM_EPOCHS = 50

# Initialize model
model = BiLSTM_CRF_Lample2016(
    vocab_size=vocab_size,
    char_vocab_size=char_vocab_size,
    embedding_dim=EMBEDDING_DIM,
    char_emb_dim=CHAR_EMB_DIM,
    char_hidden_dim=CHAR_HIDDEN_DIM,
    lstm_hidden_dim=LSTM_HIDDEN_DIM,
    num_tags=num_tags,
    embedding_matrix=embedding_matrix
).to(device)

# PAPER OPTIMIZER: SGD with momentum
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=3, verbose=True
)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())

print(f"\n🎯 LAMPLE (2016) CONFIGURATION:")
print(f"  Character encoding: BiLSTM (not CNN!)")
print(f"  Char embedding: {CHAR_EMB_DIM}d")
print(f"  Char BiLSTM hidden: {CHAR_HIDDEN_DIM} × 2 = {CHAR_HIDDEN_DIM*2}d")
print(f"  Word BiLSTM hidden: {LSTM_HIDDEN_DIM} × 2 = {LSTM_HIDDEN_DIM*2}d")
print(f"  Optimizer: SGD + momentum=0.9")
print(f"  Learning rate: {LEARNING_RATE} with decay")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Total parameters: {total_params:,}")


🎯 LAMPLE (2016) CONFIGURATION:
  Character encoding: BiLSTM (not CNN!)
  Char embedding: 25d
  Char BiLSTM hidden: 25 × 2 = 50d
  Word BiLSTM hidden: 100 × 2 = 200d
  Optimizer: SGD + momentum=0.9
  Learning rate: 0.01 with decay
  Batch size: 10
  Total parameters: 1,726,895


## 8. Training Loop

In [28]:
def train_epoch(model, data_loader, optimizer, device):
    """Training loop - empty sequences already filtered in collate_fn"""
    model.train()
    total_loss = 0
    num_batches = 0
    
    pbar = tqdm(data_loader, desc="Training", leave=False)
    
    for batch in pbar:
        # Skip empty batches (rare, but possible)
        if len(batch['lengths']) == 0:
            continue
            
        word_ids = batch['word_ids'].to(device)
        char_ids = batch['char_ids'].to(device)
        char_lens = batch['char_lens']
        tag_ids = batch['tag_ids'].to(device)
        mask = batch['mask'].to(device)
        
        optimizer.zero_grad()
        loss = model(word_ids, char_ids, char_lens, tag_ids, mask)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    return total_loss / max(num_batches, 1)


def evaluate(model, data_loader, device):
    """Evaluate - empty sequences already filtered in collate_fn"""
    model.eval()
    all_predictions = []
    all_true_tags = []
    
    with torch.no_grad():
        pbar = tqdm(data_loader, desc="Evaluating", leave=False)
        
        for batch in pbar:
            # Skip empty batches
            if len(batch['lengths']) == 0:
                continue
                
            word_ids = batch['word_ids'].to(device)
            char_ids = batch['char_ids'].to(device)
            char_lens = batch['char_lens']
            tag_ids = batch['tag_ids'].to(device)
            mask = batch['mask'].to(device)
            lengths = batch['lengths']
            
            predictions = model(word_ids, char_ids, char_lens, mask=mask)
            
            # Convert to tags
            for i, (pred, length) in enumerate(zip(predictions, lengths)):
                pred_tags = [idx2tag[idx] for idx in pred[:length]]
                true_tags = [idx2tag[tag_ids[i][j].item()] for j in range(length)]
                
                all_predictions.append(pred_tags)
                all_true_tags.append(true_tags)
    
    return all_true_tags, all_predictions

print("Training functions defined!")

Training functions defined!


In [29]:
print("Starting training with LAMPLE (2016) settings...\n")
print("=" * 80)
print("Configuration:")
print(f"  - Character encoding: BiLSTM (Lample) vs CNN (Ma & Hovy)")
print(f"  - Char BiLSTM: 25×2=50d output")
print(f"  - Word BiLSTM: 100×2=200d output")
print(f"  - Original paper result: 90.94% F1 on CoNLL-2003")
print("=" * 80 + "\n")

best_f1 = 0
patience = 5
patience_counter = 0

training_start = time.time()

for epoch in range(NUM_EPOCHS):
    epoch_start = time.time()
    
    # Train
    train_loss = train_epoch(model, train_loader, optimizer, device)
    
    # Evaluate
    val_true_tags, val_pred_tags = evaluate(model, val_loader, device)
    
    # Calculate F1
    results = evaluate_entity_spans(val_true_tags, val_pred_tags, val_tokens)
    val_f1 = results['f1']
    val_precision = results['precision']
    val_recall = results['recall']
    
    epoch_time = time.time() - epoch_start
    current_lr = optimizer.param_groups[0]['lr']
    
    print(f"Epoch {epoch+1:2d}/{NUM_EPOCHS} | "
          f"Loss: {train_loss:.4f} | "
          f"Val P: {val_precision:.4f} R: {val_recall:.4f} F1: {val_f1:.4f} | "
          f"LR: {current_lr:.5f} | "
          f"Time: {epoch_time:.1f}s")
    
    # Learning rate scheduling
    scheduler.step(val_f1)
    
    # Early stopping
    if val_f1 > best_f1:
        best_f1 = val_f1
        patience_counter = 0
        os.makedirs('models', exist_ok=True)
        torch.save(model.state_dict(), 'models/bilstm_crf_lample2016_best.pt')
        print(f"  → New best F1! Model saved.")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\nEarly stopping after {epoch+1} epochs (patience={patience})")
            break

training_time = time.time() - training_start

print("=" * 80)
print(f"\nTraining completed in {training_time:.1f}s ({training_time/60:.1f} minutes)")
print(f"Best validation F1: {best_f1:.4f}")
print(f"\nNote: This used Lample et al. (2016) with Char-BiLSTM")

Starting training with LAMPLE (2016) settings...

Configuration:
  - Character encoding: BiLSTM (Lample) vs CNN (Ma & Hovy)
  - Char BiLSTM: 25×2=50d output
  - Word BiLSTM: 100×2=200d output
  - Original paper result: 90.94% F1 on CoNLL-2003



KeyboardInterrupt: 

## 9. Load Best Model and Final Evaluation

In [ ]:
# Load best model
model.load_state_dict(torch.load('models/bilstm_crf_lample2016_best.pt'))
model.eval()

print("Best model loaded!")

# Final evaluation
val_true_tags, val_pred_tags = evaluate(model, val_loader, device)

# Comprehensive report
print_evaluation_report(
    val_true_tags,
    val_pred_tags,
    val_tokens,
    model_name="BiLSTM-CRF with Char-BiLSTM (Lample 2016)"
)

## 10. Save Model and Results

In [ ]:
# Save vocabularies
vocab_data = {
    'word2idx': word2idx,
    'char2idx': char2idx,
    'tag2idx': tag2idx,
    'idx2word': idx2word,
    'idx2char': idx2char,
    'idx2tag': idx2tag
}

with open('models/bilstm_crf_lample2016_vocab.pkl', 'wb') as f:
    pickle.dump(vocab_data, f)

print("Vocabularies saved!")

# Save results
final_results = evaluate_entity_spans(val_true_tags, val_pred_tags, val_tokens)

results_summary = {
    'model': 'BiLSTM-CRF with Char-BiLSTM (Lample et al. 2016)',
    'precision': final_results['precision'],
    'recall': final_results['recall'],
    'f1': final_results['f1'],
    'training_time': training_time,
    'num_epochs': epoch + 1,
    'hyperparameters': {
        'embedding_dim': EMBEDDING_DIM,
        'char_emb_dim': CHAR_EMB_DIM,
        'char_lstm_hidden': CHAR_HIDDEN_DIM,
        'char_output_dim': CHAR_HIDDEN_DIM * 2,
        'lstm_hidden_dim': LSTM_HIDDEN_DIM,
        'learning_rate': LEARNING_RATE,
        'batch_size': BATCH_SIZE,
        'optimizer': 'SGD with momentum=0.9',
        'lr_scheduler': 'ReduceLROnPlateau',
        'dropout': 0.5,
        'min_word_freq': MIN_WORD_FREQ
    },
    'paper_reference': 'Lample et al. (2016) - Neural Architectures for Named Entity Recognition',
    'paper_result_conll2003': '90.94% F1',
    'char_encoding': 'BiLSTM (not CNN)'
}

with open('models/bilstm_crf_lample2016_results.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

print("Results saved!")

## 11. Summary

### Lample et al. (2016) - Paper-Exact Implementation:

This notebook replicates **Lample et al. (2016)** which uses:

✅ **Character encoding**: BiLSTM (25d hidden × 2 directions = 50d output)  
✅ **Word BiLSTM**: 100 hidden × 2 = 200d output  
✅ **Optimizer**: SGD + momentum=0.9  
✅ **Learning rate**: 0.01 with decay  
✅ **Batch size**: 10  

### Comparison of Three Implementations:

| Feature | M4.ipynb | M4_Paper_Exact.ipynb | M4_Lample2016.ipynb |
|---------|----------|---------------------|--------------------|
| **Char encoding** | CNN (30d) | CNN (30d) | **BiLSTM (50d)** |
| **LSTM hidden** | 256 | 100 | 100 |
| **Optimizer** | Adam | SGD | SGD |
| **Batch size** | 32 | 10 | 10 |
| **Paper** | — | Ma & Hovy 2016 | Lample 2016 |
| **Paper F1** | — | 91.21% | 90.94% |

### Key Difference: Character Encoding

**Lample (2016)**: Uses BiLSTM to process character sequences
- Pro: Can capture longer-range character patterns
- Con: Slower (sequential processing)

**Ma & Hovy (2016)**: Uses CNN to process characters
- Pro: Faster (parallel processing)
- Con: Limited receptive field (kernel size 3)

### Expected Results:

**Paper (CoNLL-2003)**: 90.94% F1  
**Your dataset**: 80-85% F1 (harder - 15 entity types)

### Which is Better?

Both papers achieved similar results (~91% F1 on CoNLL-2003):  
- **Ma & Hovy (CNN)**: 91.21% F1  
- **Lample (BiLSTM)**: 90.94% F1  

CNN is slightly better and faster, which is why it became more popular!

### Reference:

**Lample, G., Ballesteros, M., Subramanian, S., Kawakami, K., & Dyer, C. (2016)**. Neural architectures for named entity recognition. In Proceedings of NAACL 2016.
- Paper: https://arxiv.org/abs/1603.01360
- Result: 90.94% F1 on CoNLL-2003 English NER
- Key innovation: Character-level BiLSTM for word representations